In [ ]:
using JuMP
using HiGHS  # You can replace HiGHS with OSQP or another solver of your choice
using LinearAlgebra

# Define the number of particles
L = 100  # Number of particles

# Problem dimensions
n = 5    # Dimension of the control variable (u)

# Generate shared problem data
Q = Matrix{Float64}(I, n, n)   # Quadratic cost matrix (identity matrix)
c = randn(n,L)                   # Linear cost vector (randomized)
A = vcat(ones(1, n), -ones(1, n))  # Inequality constraint matrix
b = [1.0; -1.0]                # Inequality constraint bounds

# Define a function to solve the QP for a single particle
function solve_qp_for_particle(i,c_tot)
    model = Model(HiGHS.Optimizer)  # Create a JuMP model with HiGHS solver

    # Define variables
    @variable(model, u[1:n])

    c = c_tot[:,i]

    # Define the objective function
    @objective(model, Min, 0.5 * u' * Q * u + c' * u)

    # Define constraints
    @constraint(model, A * u .<= b)

    # Solve the optimization problem
    optimize!(model);

    # Check solver status and return results
    if termination_status(model) == MOI.OPTIMAL
        return value.(u)
    else
        println("Particle $i: Solution not optimal.")
        return nothing
    end
end

# Solve the QP for all particles in parallel
results = Vector{Union{Vector{Float64}, Nothing}}(undef, L)  # Store results for each particle
for i in 1:L
    results[i] = solve_qp_for_particle(i,c)
end



Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [4e-01, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
  Iteration        Objective     NullspaceDim
          0      -0.34178719                4      0.00s
          1       -1.6506675                4      0.00s
Model status        : Optimal
QP ASM    iterations: 1
Objective value     : -1.6506675836e+00
HiGHS run time      :          0.00
Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e-01, 2e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
  Iteration        Objective     NullspaceDim
          0        2.4803615                4      0.00s
          3       -2.2425099                4      0.00s
Model status        : Optimal
QP ASM    iterations: 3
Objective value     : -2.2425102067e+00
HiGHS run time      :          0.00
Runn

In [ ]:
# Print results for the first few particles
println("First few results:")
for i in 1:min(5, L)
    println("Particle $i solution: ", results[i])
end
